# Neural Network using PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader

In [17]:
# Define the Neural Network Architecture
class SimpleNN(nn.Module):
    """
    A simple feedforward neural network for image classification.
    """
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, num_classes)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.network(x)
        return logits

In [18]:
# Setup the hyperparameters and devices
INPUT_SIZE = 784
HIDDEN_SIZE1 = 128
HIDDEN_SIZE2 = 64
NUM_CLASSES = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 64
NUM_EPOCHS = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
# Load and prepare the dataset

# Define a transform to normalize the data
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))
])

# Download and load the dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:03<00:00, 3.24MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 112kB/s]
100%|██████████| 1.65M/1.65M [00:05<00:00, 280kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.34MB/s]


In [13]:
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [20]:
model = SimpleNN(INPUT_SIZE, HIDDEN_SIZE1, HIDDEN_SIZE2, NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [24]:
# Train the network
print("\nStarting training...")

for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i+1}/{len(train_loader)//BATCH_SIZE}], Loss: {loss.item():.4f}')


Starting training...
Epoch [1/5], Step [100/14], Loss: 0.3619
Epoch [1/5], Step [200/14], Loss: 0.3149
Epoch [1/5], Step [300/14], Loss: 0.2866
Epoch [1/5], Step [400/14], Loss: 0.3520
Epoch [1/5], Step [500/14], Loss: 0.1732
Epoch [1/5], Step [600/14], Loss: 0.2120
Epoch [1/5], Step [700/14], Loss: 0.3522
Epoch [1/5], Step [800/14], Loss: 0.1645
Epoch [1/5], Step [900/14], Loss: 0.2065
Epoch [2/5], Step [100/14], Loss: 0.2588
Epoch [2/5], Step [200/14], Loss: 0.3542
Epoch [2/5], Step [300/14], Loss: 0.1131
Epoch [2/5], Step [400/14], Loss: 0.0860
Epoch [2/5], Step [500/14], Loss: 0.0829
Epoch [2/5], Step [600/14], Loss: 0.1375
Epoch [2/5], Step [700/14], Loss: 0.1450
Epoch [2/5], Step [800/14], Loss: 0.0850
Epoch [2/5], Step [900/14], Loss: 0.0692
Epoch [3/5], Step [100/14], Loss: 0.1246
Epoch [3/5], Step [200/14], Loss: 0.0661
Epoch [3/5], Step [300/14], Loss: 0.0758
Epoch [3/5], Step [400/14], Loss: 0.0903
Epoch [3/5], Step [500/14], Loss: 0.0464
Epoch [3/5], Step [600/14], Loss: 0

In [16]:
print("\nEvaluating model on test data...")
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy:.2f}%')


Evaluating model on test data...
Accuracy of the model on the test images: 96.75%
